# CNMF Componenent evaluation

This is similar to the last visualization in `cnmf_viz.ipynb` but combines it with component evaluation

View cmaps with `r_values`, `cnn_preds`, `comp_SNR`

Click on components and press `"g"` to add to good, `"b"` to add to bad.

In [1]:
from mesmerize_core import *
import numpy as np
import pandas as pd

2023-01-08 02:03:38.940651: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 02:03:39.023944: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-08 02:03:39.415634: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kushalk/python-venvs/mescore/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/lib64
2023-01-08 02:03:39.415684: W tensorflow/stream_executor/platform/default/dso_

In [13]:
from fastplotlib import ImageWidget, Plot, GridPlot
from fastplotlib.utils import get_colors
from fastplotlib.graphics.line_slider import LineSlider
from ipywidgets import VBox, IntSlider, Layout

ImportError: cannot import name 'make_colors' from 'fastplotlib.utils' (/home/kushalk/repos/fastplotlib/fastplotlib/utils/__init__.py)

In [3]:
pd.options.display.max_colwidth = 120

# Paths

In [4]:
# for this demo set this dir as the path to your `caiman_data` dir
set_parent_raw_data_path("/home/kushalk/caiman_data/")

# batch path
batch_path = get_parent_raw_data_path().joinpath("mesmerize-core-batch/batch.pickle")

## Load batch

In [5]:
df = load_batch(batch_path)
df

,algo,item_name,input_movie_path,params,outputs,comments,uuid
0,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (6, 6), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan': '...",{'mean-projection-path': 09610b86-917a-41e7-a493-2d036bf44428/09610b86-917a-41e7-a493-2d036bf44428_mean_projection.n...,None,09610b86-917a-41e7-a493-2d036bf44428
1,cnmf,Sue_2x_3000_40_-46,09610b86-917a-41e7-a493-2d036bf44428/09610b86-917a-41e7-a493-2d036bf44428-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (4, 4), 'ssub': 1, 't...",{'mean-projection-path': 2b9dabd8-7d22-4327-8479-aea36bde79cf/2b9dabd8-7d22-4327-8479-aea36bde79cf_mean_projection.n...,None,2b9dabd8-7d22-4327-8479-aea36bde79cf
2,cnmf,Sue_2x_3000_40_-46,09610b86-917a-41e7-a493-2d036bf44428/09610b86-917a-41e7-a493-2d036bf44428-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (6, 6), 'ssub': 1, 'ts...",{'mean-projection-path': 38f85afb-417f-453a-8da5-28753b0dc290/38f85afb-417f-453a-8da5-28753b0dc290_mean_projection.n...,None,38f85afb-417f-453a-8da5-28753b0dc290
3,cnmf,Sue_2x_3000_40_-46,09610b86-917a-41e7-a493-2d036bf44428/09610b86-917a-41e7-a493-2d036bf44428-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (6, 6), 'ssub': 1, 't...",{'mean-projection-path': a49689ad-f093-49ff-9a86-058835eccd34/a49689ad-f093-49ff-9a86-058835eccd34_mean_projection.n...,None,a49689ad-f093-49ff-9a86-058835eccd34
4,cnmf,Sue_2x_3000_40_-46,09610b86-917a-41e7-a493-2d036bf44428/09610b86-917a-41e7-a493-2d036bf44428-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (6, 6), 'ssub': 1, 'ts...",{'mean-projection-path': f67dbcb9-fd5b-4d81-acc4-f080a7f81db4/f67dbcb9-fd5b-4d81-acc4-f080a7f81db4_mean_projection.n...,None,f67dbcb9-fd5b-4d81-acc4-f080a7f81db4
5,cnmf,Sue_2x_3000_40_-46,09610b86-917a-41e7-a493-2d036bf44428/09610b86-917a-41e7-a493-2d036bf44428-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (6, 6), 'ssub': 1, 't...",{'mean-projection-path': ff76c624-2c29-4c2d-ba91-3eb471a8e9e3/ff76c624-2c29-4c2d-ba91-3eb471a8e9e3_mean_projection.n...,None,ff76c624-2c29-4c2d-ba91-3eb471a8e9e3
6,cnmf,Sue_2x_3000_40_-46,09610b86-917a-41e7-a493-2d036bf44428/09610b86-917a-41e7-a493-2d036bf44428-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (8, 8), 'ssub': 1, 'ts...",{'mean-projection-path': 879d1d8b-dfe7-4b4f-aa2f-b987d83f7788/879d1d8b-dfe7-4b4f-aa2f-b987d83f7788_mean_projection.n...,None,879d1d8b-dfe7-4b4f-aa2f-b987d83f7788
7,cnmf,Sue_2x_3000_40_-46,09610b86-917a-41e7-a493-2d036bf44428/09610b86-917a-41e7-a493-2d036bf44428-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (8, 8), 'ssub': 1, 't...",{'mean-projection-path': 539745c5-ec3e-4a63-a7c2-eb41af9a080e/539745c5-ec3e-4a63-a7c2-eb41af9a080e_mean_projection.n...,None,539745c5-ec3e-4a63-a7c2-eb41af9a080e
8,cnmf,Sue_2x_3000_40_-46,09610b86-917a-41e7-a493-2d036bf44428/09610b86-917a-41e7-a493-2d036bf44428-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (8, 8), 'ssub': 1, 'ts...",{'mean-projection-path': da760704-f760-4f34-b26a-06eb5cc3cfec/da760704-f760-4f34-b26a-06eb5cc3cfec_mean_projection.n...,None,da760704-f760-4f34-b26a-06eb5cc3cfec
9,cnmf,Sue_2x_3000_40_-46,09610b86-917a-41e7-a493-2d036bf44428/09610b86-917a-41e7-a493-2d036bf44428-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'

In [10]:
# You can change this to plot the outputs for different batch items
index = 1

# get the motion corrected input movie as a memmap
cnmf_movie = df.iloc[index].caiman.get_input_movie()

# we can get the contours of the spatial components
contours, coms = df.iloc[index].cnmf.get_contours("all", swap_dim=False)

# and temporal components
temporal = df.iloc[index].cnmf.get_temporal("all")

ixs_good = df.iloc[index].cnmf.get_good_components()
ixs_bad = df.iloc[index].cnmf.get_bad_components()

cnmf_obj = df.iloc[index].cnmf.get_output()

Decode mmap filename /home/kushalk/caiman_data/mesmerize-core-batch/09610b86-917a-41e7-a493-2d036bf44428/09610b86-917a-41e7-a493-2d036bf44428-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap


In [12]:
np.sort(cnmf_obj.estimates.r_values)

array([-0.19361448, -0.00949648,  0.00407439,  0.00752039,  0.04555327,
        0.06338374,  0.14284759,  0.17890952,  0.18812357,  0.20339994,
        0.20821096,  0.24875144,  0.28352673,  0.30002904,  0.31564671,
        0.32751131,  0.34441391,  0.36287948,  0.37505725,  0.37656569,
        0.37759551,  0.38364631,  0.38383077,  0.39639933,  0.40045328,
        0.41895777,  0.44048263,  0.4434438 ,  0.44841094,  0.44913498,
        0.45965439,  0.47577601,  0.47721031,  0.48324347,  0.48630648,
        0.48822328,  0.50031794,  0.5029086 ,  0.50680755,  0.50682535,
        0.50868848,  0.51604714,  0.51657431,  0.51796069,  0.51894883,
        0.52217412,  0.53642036,  0.54972745,  0.54997033,  0.55028392,
        0.55152912,  0.55249477,  0.55396109,  0.55498617,  0.55963035,
        0.56667794,  0.56816887,  0.58273237,  0.59273873,  0.59968857,
        0.60124371,  0.60183431,  0.60692199,  0.62156763,  0.62401282,
        0.6252982 ,  0.63185618,  0.63543767,  0.63690626,  0.64

In [9]:
rcm = df.iloc[index].cnmf.get_rcm()
corr_img = df.iloc[index].caiman.get_corr_image()

In [ ]:
# 2 x 2 gridplot
# |=====================|
# |  movie |    rcm     |
# |---------------------|
# | corr  |  temporal   |
# |=====================|
cnmf_grid = GridPlot(
    shape=(2, 2), 
    controllers=[[0, 0], [0, 1]], 
    names=[["movie", "rcm"], ["corr", "temporal"]]
)

# movie and rcm, rcm is a lazy array and behaves similar to numpy arrays
movie_graphic = cnmf_grid["movie"].add_image(cnmf_movie[0], cmap="gnuplot2")
rcm_graphic = cnmf_grid["rcm"].add_image(rcm[0], cmap="gnuplot2")

# corr image
cnmf_grid["corr"].add_image(corr_img, cmap="viridis")

# contours for good components
contours_good, coms = df.iloc[index].cnmf.get_contours("good", swap_dim=False)

# random colors for contours and temporal components
# make an RGBA array for each color
rand_colors = np.random.rand(len(contours_good), 4)  # [n_contours, RGBA]
rand_colors[:, -1] = 1 # set alpha = 1

# get temporal of only good components
temporal_good = df.iloc[index].cnmf.get_temporal("good")

# add contours to both movie and rcm subplots
contours_movie = cnmf_grid["movie"].add_line_collection(contours_good, colors=rand_colors)
contours_rcm = cnmf_grid["rcm"].add_line_collection(contours_good, colors=rand_colors)

# line stack of temporal components
temporal_stack = cnmf_grid["temporal"].add_line_stack(temporal_good, colors=rand_colors, thickness=3.0, separate=15)

# plot single temporal, just like before
plot_temporal_single = Plot()
temporal_graphic = plot_temporal_single.add_line_collection(temporal_good, colors=rand_colors)

# since this is a GridPlot and not an ImageWidget we need to define sliders
slider = IntSlider(min=0, max=cnmf_movie.shape[0] - 1, value=0, step=1)

# vertical line sliders
_ls = LineSlider(x_pos=0, bounds=(temporal.min(), temporal.max()), slider=slider)
_ls2 = LineSlider(x_pos=0, bounds=(temporal.min(), temporal.max() + temporal_stack.items[-1].position.y), slider=slider)
plot_temporal_single.add_graphic(_ls)
cnmf_grid["temporal"].add_graphic(_ls2)

# function to update each frame
def update_frame(change):
    ix = change["new"]
    movie_graphic.data = cnmf_movie[ix]
    rcm_graphic.data = rcm[ix]
    
slider.observe(update_frame, "value")

@plot_temporal_single.renderer.add_event_handler("resize")
def update_slider_width(*args):
    width, h = plot_temporal_single.renderer.logical_size
    slider.layout = Layout(width=f"{width}px")
    
VBox([plot_temporal_single.show(), cnmf_grid.show(), slider])